In [ ]:
import theano
from theano import tensor as T
import numpy as np
import matplotlib.pyplot as plt
from load import mnist

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * .001))

def model(X,w):
    return T.nnet.argmax(T.dot(X,w))

trX, trY, teX, teY = mnist(onehot=True)
X = T.fmatrix()
Y = T.fmatrix()

w = init_weights((784, 10))

py_x = model(X, w)
y_pred = T.argmax(py_x, axis = 1)

cost = T.mean(T.nnet.categorical.crossentropy(py_x, Y))
gradient = T.grad(cost = cost, wrt = w)
updates = [[w, w - gradient * 0.05]]

train = theano.function(inputs=[X, Y], outputs = cost, updates = updates, allow_input_downcast = True)
predict = theano.function(inputs=[X], output = y_pred, allow_input_downcast = True)

for i in range (100):
    for start, end in zip(range(0, len(trX), 128), range(0, len(trX), 128)):
        cost = train(trX[start:end], trY[start:end])
    print i, np.mean(np.argmax(teY, axis=1) == predict(teX))